In [ ]:
import gymnasium as gym
import numpy as np
np.random.seed(42)

In [ ]:
env = gym.make("MountainCar-v0")
#env = gym.make("MountainCar-v0" , render_mode="human")
#env.reset()
env.close()
# done = False
# while not done:
#     action = 2  # always go right!
#     env.step(action)
#     env.render()

In [ ]:
env.observation_space
#a)first array is min value for position and velocity
#b)second array is max value for position and velocity

In [ ]:
for i in range(10):
    res=env.action_space.sample()
    print(f"{i}-th sampling gave {res} as the action")
    

In [ ]:
print(env.observation_space.high) #max-min position of distance and max-min value of velocity
print(env.observation_space.low)

In [ ]:
dis = 20 #poso tha "temaxistei" to observation kai action space
DISCRETE_OS_SIZE = [dis, dis]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE
print(discrete_os_win_size)

In [ ]:
0.09*20 #is equal to [-1.2,0.6]

# Q-Table


In [ ]:
# Q-Learning settings
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 4*25001
EPISODES

In [ ]:
#greedy epsilon
# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
#high epsilon -> agent prefers exploration
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2 #apo ayto to epeisodio kai meta to e = 0 = constant
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)
END_EPSILON_DECAYING

In [ ]:
EPISODES//2

In [ ]:
# epsilon_old = []
# epsilon=1
# for ep in range(EPISODES):
#     epsilon_old=np.append(epsilon_old, epsilon)
#     if END_EPSILON_DECAYING >= ep >= START_EPSILON_DECAYING:
            
#             epsilon -= epsilon_decay_value
# len(epsilon_old[epsilon_old < 0])  
# #len(epsilon_old)
# # #output = 0 

In [ ]:
# epsilon_old

In [ ]:
# len(np.unique(epsilon_old)) == len(epsilon_old) # some episodes have same epsilon
# #output = False

In [ ]:
(DISCRETE_OS_SIZE + [env.action_space.n])
#20 possible x positions
#20 possible velocity values
#3 possible actions

In [ ]:
def get_discrete_state(state):
    if type(state) == tuple: #if epeidh to env.reset() epistrefei tuple, enw to env.step()[0] epistrefei array
        #to new_state
        state = state[0]
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    #return tuple(discrete_state.astype(int)) #older version?
    return tuple(discrete_state.astype(np.int64))

In [ ]:
same_env_reset=env.reset()
print(f"state discrete: {get_discrete_state(same_env_reset)} \nenv output :{same_env_reset}")

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
q_table.shape

In [ ]:
random_state=env.reset()
q_table[get_discrete_state(random_state)] #epistrefei to Q-value gia ta 3 actions
#o agent epilegei to megisto Q-value
print(f"q-table={q_table[get_discrete_state(random_state)]} \nargmax = {np.argmax(q_table[get_discrete_state(random_state)])}")

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
for ep in range(EPISODES):
    current_state = get_discrete_state(env.reset())
    i=0 #agent's moves counter
    done=False #it becomes true from env.step()
    while not done:
        #choose best action for current_state
        action=np.argmax(q_table[current_state])
        #after action, we have a new state and the reward for the action that moved us "forward"
        new_state, reward, done, truncated , information = env.step(action)
        new_state_x=new_state[0] #first value is x position #check x position for env.goal_position
        new_state=get_discrete_state(new_state) #discretize our new state
        #Qmax: apo ton mh ananewmeno pinaka, vriskw to megisto Q gia to 
        Q_max = np.max(q_table[new_state])
        #Q: apo thn mhn ananewmeno pinaka, to Q to gia state kai to action poy phra
        Q_current=q_table[current_state + (action,)]
        #eq
        new_q=(1 - LEARNING_RATE) * Q_current + LEARNING_RATE * (reward + DISCOUNT * Q_max)
        #update Q table
        q_table[current_state + (action,)] = new_q        
        if new_state_x >= env.goal_position:
            q_table[current_state + (action,)]=0
            #print(f"megalo X")
        if done==True and ep%5000==0:
            print(f"Done at {i}-th move durin {ep}-th episode.")
        else:
            i=i+1
        current_state=new_state

#output
# Done at 13516-th move durin 0-th episode.
# Done at 151-th move durin 5000-th episode.
# Done at 124-th move durin 10000-th episode.
# Done at 107-th move durin 15000-th episode.
# Done at 118-th move durin 20000-th episode.
# Done at 107-th move durin 25000-th episode.
# Done at 109-th move durin 30000-th episode.
# Done at 110-th move durin 35000-th episode.
# Done at 110-th move durin 40000-th episode.
# Done at 110-th move durin 45000-th episode.
# Done at 110-th move durin 50000-th episode.
# Done at 107-th move durin 55000-th episode.
# Done at 173-th move durin 60000-th episode.
# Done at 109-th move durin 65000-th episode.
# Done at 108-th move durin 70000-th episode.
# Done at 108-th move durin 75000-th episode.
# Done at 108-th move durin 80000-th episode.
# Done at 175-th move durin 85000-th episode.
# Done at 175-th move durin 90000-th episode.
# Done at 108-th move durin 95000-th episode.
# Done at 108-th move durin 100000-th episode.

In [ ]:
len(q_table[q_table>0])

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
epsilon=1
for ep in range(EPISODES):
    current_state = get_discrete_state(env.reset())
    i=0 #agent's moves counter
    done=False #it becomes true from env.step()
    while not done:
        if np.random.random() > epsilon:
        #choose best action for current_state
            action=np.argmax(q_table[current_state])
        else:
            action=np.random.randint(0, env.action_space.n)
        #after action, we have a new state and the reward for the action that moved us "forward"
        new_state, reward, done, truncated , information = env.step(action)
        new_state_x=new_state[0] #first value is x position #check x position for env.goal_position
        new_state=get_discrete_state(new_state) #discretize our new state
        #Qmax: apo ton mh ananewmeno pinaka, vriskw to megisto Q gia to 
        Q_max = np.max(q_table[new_state])
        #Q: apo thn mhn ananewmeno pinaka, to Q to gia state kai to action poy phra
        Q_current=q_table[current_state + (action,)]
        #eq
        new_q=(1 - LEARNING_RATE) * Q_current + LEARNING_RATE * (reward + DISCOUNT * Q_max)
        #update Q table
        q_table[current_state + (action,)] = new_q        
        if new_state_x >= env.goal_position:
            q_table[current_state + (action,)]=0
            #print(f"megalo X")
        if done==True and ep%5000==0:
            print(f"Done with {i}-th move during {ep}-th episode. Epsilon={epsilon}")
        else:
            i=i+1
        current_state=new_state
    if END_EPSILON_DECAYING >= ep >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value
        #print(f"epsilon value = {epsilon}")
    
#output
# Done with 16547-th move during 0-th episode. Epsilon=1
# Done with 4826-th move during 5000-th episode. Epsilon=0.9000219995601054
# Done with 1088-th move during 10000-th episode. Epsilon=0.8000239995202029
# Done with 887-th move during 15000-th episode. Epsilon=0.7000259994803003
# Done with 592-th move during 20000-th episode. Epsilon=0.6000279994403978
# Done with 467-th move during 25000-th episode. Epsilon=0.5000299994004952
# Done with 638-th move during 30000-th episode. Epsilon=0.40003199936059264
# Done with 159-th move during 35000-th episode. Epsilon=0.3000339993206901
# Done with 157-th move during 40000-th episode. Epsilon=0.20003599928071816
# Done with 161-th move during 45000-th episode. Epsilon=0.10003799924069416
# Done with 113-th move during 50000-th episode. Epsilon=3.9999200701827236e-05
# Done with 135-th move during 55000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 151-th move during 60000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 144-th move during 65000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 148-th move during 70000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 153-th move during 75000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 142-th move during 80000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 105-th move during 85000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 144-th move during 90000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 154-th move during 95000-th episode. Epsilon=-1.9999599322172286e-05
# Done with 138-th move during 100000-th episode. Epsilon=-1.9999599322172286e-05

In [ ]:
goal_position=env.goal_position
max_speed=env.observation_space.high[1]
min_speed=env.observation_space.low[1]
velocity=np.random.uniform(min_speed,max_speed)
goal_state=(goal_position,velocity)
get_discrete_state(goal_state)

In [ ]:
max_position=env.observation_space.high[0]
min_position=env.observation_space.low[0]
print(f"max position:{max_position}, min position:{min_position}")

In [ ]:
get_discrete_state((max_position, velocity))

In [ ]:
max_speed=env.observation_space.high[1]
min_speed=env.observation_space.low[1]
np.random.uniform(min_speed,max_speed)

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
discrete_state=get_discrete_state(env.reset())
done=False
i=0
while not done:

    action = np.argmax(q_table[discrete_state])
    new_state, reward, done, _, _ = env.step(action)
    #print(done)
    new_discrete_state = get_discrete_state(new_state)

    #env.render()
    #new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

    # If simulation did not end yet after last step - update Q table
    if not done:

        # Maximum possible Q value in next step (for new state)
        max_future_q = np.max(q_table[new_discrete_state])

        # Current Q value (for current state and performed action)
        current_q = q_table[discrete_state + (action,)]

        # And here's our equation for a new Q value for current state and action
        new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

        # Update Q table with new Q value
        q_table[discrete_state + (action,)] = new_q


#     # Simulation ended (for any reson) - if goal position is achived - update Q value with reward directly
#     elif new_state[0] >= env.goal_position:
#         #q_table[discrete_state + (action,)] = reward
#         q_table[discrete_state + (action,)] = 0
    print(f"Done: {done}, i-th move:{i}")
    i=i+1
    discrete_state = new_discrete_state


#env.close()